In [1]:
import pennylane as qml
import pennylane.numpy as np
import torch
from torch.nn import MSELoss, CrossEntropyLoss
from torch.optim import Adam, Adagrad
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset
from torch.utils.data import random_split


import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [2]:
EPOCHS = 10
SEED = 16
TRAINSIZE = 10000
VALSIZE = 1000
TESTSIZE = 1000

In [3]:
from QCNN_circuit_only import QCNN
from ansatz import ConvCirc1, PoolingCirc

INPUT_QUBITS = 8

qcnn = QCNN(n_qubits=INPUT_QUBITS, conv_ansatz=ConvCirc1(), pooling_ansatz=PoolingCirc(), stride=1)

dev = qml.device('default.qubit', wires=qcnn.total_qubit)

@qml.qnode(dev, interface='torch')
def circuit(params, data):
    qcnn.construct_circuit(params, data)
    return qml.expval(qml.PauliZ(0))


In [4]:
from datasetloading import TrainTestLoader

loader = TrainTestLoader('./mnist_train.csv', './mnist_test.csv')
loader.scale_pca(n_dim=30)
training_set, test_set = loader.return_dataset()
print('PCA Complete')

training_set.filter([0, 1])
training_set.periodic_data_padding()
print('Training Data Processing Complete')
test_set.filter([0, 1])
test_set.periodic_data_padding()
print('Test Data Processing Complete')

training_set, validation_set = random_split(training_set, [TRAINSIZE, len(training_set)-TRAINSIZE], generator=torch.Generator().manual_seed(SEED))
validation_set, _ = random_split(validation_set, [VALSIZE, len(validation_set)-VALSIZE], generator=torch.Generator().manual_seed(SEED))
test_set, _ = random_split(test_set, [TESTSIZE, len(test_set)-TESTSIZE], generator=torch.Generator().manual_seed(SEED))

# Create data loaders for our datasets; shuffle for training, not for test
training_loader = DataLoader(training_set, batch_size=25, shuffle=True, generator=torch.Generator().manual_seed(SEED))
validation_loader = DataLoader(validation_set, batch_size=25, shuffle=False, generator=torch.Generator().manual_seed(SEED))
test_loader = DataLoader(test_set, batch_size=25, shuffle=False, generator=torch.Generator().manual_seed(SEED))

# ====================================

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('validation set has {} instances'.format(len(validation_set)))
print('test set has {} instances'.format(len(test_set)))

PCA Complete
Training Data Processing Complete
Test Data Processing Complete
Training set has 10000 instances
validation set has 1000 instances
test set has 1000 instances


/home/quic/Ju-Young Ryu/PeachIceTea-1/datasetloading.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.data[i] = self.data[int((num_added_columns % data_dim))]


In [5]:
torch.manual_seed(SEED)
loss_fn = MSELoss() # TODO: choose loss function
params = torch.rand(qcnn.Calculate_Param_Num(), requires_grad=True)

optimizer = Adam([params], lr=0.01) # TODO: choose optimizer

In [6]:
import datetime
from pathlib import Path

loggername = f'{str(datetime.datetime.now())}'
filepath = Path(f'./experiment_results/{loggername}')
filepath.mkdir(parents=True, exist_ok=True)

def estimated_label(params, data):
    return circuit(params, data) # TODO: customize your extimated label

writer = SummaryWriter(log_dir='./PeriodicPadding')

loss_list = []
avg_loss_list = []
avg_vloss_list = []
history_of_parameters = []
for epoch in range(EPOCHS):
    avg_loss = 0.0
    for i, data in enumerate(training_loader):
        inputs, labels = data
        optimizer.zero_grad()
        loss = torch.tensor(0, dtype=torch.float64)
        for input, label in zip(inputs, labels):
            output = estimated_label(params, input.to(torch.float64))
            loss+=loss_fn(output, 2*label.to(torch.float64)-1)/len(labels)
        loss.backward()       
        optimizer.step()
        avg_loss += loss.item()
        loss_list.append(loss.item())
        writer.add_scalar('local training loss', loss.item(), epoch*len(training_loader)+i)
        np.save(filepath/f'epoch{epoch}_iter{epoch*len(training_loader)+i}', params.detach().numpy())
        # logger.debug("epoch: {} batch_index: {} --- loss: {} \t params: {}".format(epoch+1, i+1, loss.item(), params.detach().numpy()))


    avg_loss = avg_loss/len(training_loader)
    avg_loss_list.append(avg_loss)
    
    
    with torch.no_grad():
        avg_vloss = 0.0
        for i, vdata in enumerate(validation_loader):
            vloss = torch.tensor(0, dtype=torch.float64)
            vinputs, vlabels = vdata
            for vinput, vlabel in zip(vinputs, vlabels):
                voutput = estimated_label(params, vinput.to(torch.float64))
                vloss += loss_fn(voutput, 2*vlabel.to(torch.float64)-1)/len(vlabels)
            avg_vloss += vloss.item()
        avg_vloss = avg_vloss/len(validation_loader)
        avg_vloss_list.append(avg_vloss)
        history_of_parameters.append(params.numpy())
        np.save(filepath/f'vaidataion_epoch{epoch}', params.numpy())
    # print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # Log the running loss averaged per batch
    # for both training and test
    writer.add_scalar('validation Loss', avg_vloss, epoch)
    writer.flush()
        # print('{}: {}'.format(i, loss.item()))

/home/quic/anaconda3/envs/ddi/lib/python3.9/site-packages/torch/autograd/__init__.py:154: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180549130/work/aten/src/ATen/native/Copy.cpp:244.)
  Variable._execution_engine.run_backward(


In [7]:
# Calcaulate Test Accuracy
est_labels = []
true_labels = []
for i, data in enumerate(test_loader):
    test_datas, labels = data
    for label, test_data in zip(labels, test_datas):
        est_labels.append(estimated_label(params, test_data)>0)
        true_labels.append(label)

accuracy = sum(np.array(est_labels)==np.array(true_labels))/len(est_labels)
print(accuracy)

0.946
